<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Forming a multi-stock portfolio

In [1]:
import pandas_datareader.data as web
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline
p = print

In [2]:
# Create a list of stocks
tickers = ['AAPL','IBM', 'MSFT']

In [3]:
# # Create a dataframe of stock prices
# df = web.DataReader(tickers,'yahoo','01/01/2010','10/18/2019')['Adj Close']

In [4]:
# # Save the data to a file
# df.to_csv('AAPL_MSFT_IBM.csv')

In [5]:
# load the stock data
df = pd.read_csv('AAPL_MSFT_IBM.csv', index_col='Date', parse_dates=True)

In [6]:
# Print the full summary
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2466 entries, 2010-01-04 to 2019-10-18
Data columns (total 3 columns):
AAPL    2466 non-null float64
IBM     2466 non-null float64
MSFT    2466 non-null float64
dtypes: float64(3)
memory usage: 77.1 KB


In [7]:
# Print the first few rows
df.head()

,AAPL,IBM,MSFT
Date,,,
2010-01-04,26.601469,96.997330,24.360727
2010-01-05,26.647457,95.825584,24.368599
2010-01-06,26.223597,95.203102,24.219046
2010-01-07,26.175119,94.873573,23.967175
2010-01-08,26.349140,95.825584,24.132469


In [8]:
# Print the last few rows
df.tail()

,AAPL,IBM,MSFT
Date,,,
2019-10-14,235.163971,140.381943,139.076767
2019-10-15,234.615631,141.330734,141.089920
2019-10-16,233.668457,140.451126,139.933838
2019-10-17,234.575745,132.692764,139.216278
2019-10-18,235.702362,132.524750,136.944016


In [9]:
# Calculate the daily returns
returns = df.pct_change().dropna()

In [10]:
# Print the first few rows
returns.head()

,AAPL,IBM,MSFT
Date,,,
2010-01-05,0.001729,-0.012080,0.000323
2010-01-06,-0.015906,-0.006496,-0.006137
2010-01-07,-0.001849,-0.003461,-0.010400
2010-01-08,0.006648,0.010035,0.006897
2010-01-11,-0.008821,-0.010470,-0.012720


In [11]:
# Print the last rows
returns.tail()

,AAPL,IBM,MSFT
Date,,,
2019-10-14,-0.001439,-0.005043,-0.000931
2019-10-15,-0.002332,0.006759,0.014475
2019-10-16,-0.004037,-0.006224,-0.008194
2019-10-17,0.003883,-0.055239,-0.005128
2019-10-18,0.004803,-0.001266,-0.016322


In [12]:
# Create a vector of asset weights
weights = np.asarray([0.5, 0.3, 0.2])

In [13]:
# Hypothetical risk-free rate of retutn
rf = 0.0003

In [14]:
# Create a function to analyse portfolio performance
def portofolio_performance(returns, weights):
    # Calculate the mean, standard deviatiatio and covariance of returns
    mean, std, cov_matrix = returns.mean(), returns.std(), returns.cov()
    
    # Calculate portfolio return
    pf_ret = weights.T.dot(mean * 252)
    
    # Calculate portfolio volatiltiy
    pf_vol = np.sqrt(weights.T.dot(cov_matrix * 252).dot(weights))
    
    # Calculate portfolio Sharpe ratio
    pf_sharpe = (pf_ret - rf) / pf_vol
    
    # Make a list of portfolio return, volatility, Sharpe ratio, and and asset names
    columns = ['Ret','Vol','Sharpe'] + [ticker for ticker in tickers]
    
    # Make a list of portfolio return, volatility, Sharpe ratio, and asset weight values
    data = [pf_ret,pf_vol,pf_sharpe] + [weight for weight in weights]
    
    # Create a table of data with labeled columns 
    results = pd.Series(data=data, index=columns)
    
    # Return the portfolio performance
    return results

In [15]:
# Calculate portfolio performace
pf = portofolio_performance(returns, weights)

In [16]:
# Print the portfolio performance
pf

Ret       0.184298
Vol       0.190583
Sharpe    0.965448
AAPL      0.500000
IBM       0.300000
MSFT      0.200000
dtype: float64

We want to investigate the portfolio poerformance using different asset weights. Maually changing each weight would be tedious. Instead, we will randomly select the asset weights of each portfolio, then compare the portfolios.

In [17]:
# Set the random number generator
np.random.seed(123)

# Determine the number of portfolios
n_portfolio = 5

# Create an empty list of portoflio weights
pf_weights = []

# Create portoflios with randomly weighted assets
for i in range(n_portfolio):
        
        # Randomly generate a weight for each asset
        weights = np.random.random(len(tickers))
        
        # Normalize the weights to set the sum to 1
        weights /= sum(weights)
        
        # Attach each weights vecotr to the list of portfolio weights
        pf_weights.append(weights)

In [18]:
# Print the list of weights vectors
pf_weights

[array([0.57585137, 0.23658438, 0.18756425]),
 array([0.32547255, 0.42474357, 0.24978388]),
 array([0.45690771, 0.31904099, 0.2240513 ]),
 array([0.26777669, 0.23435595, 0.49786737]),
 array([0.48931718, 0.06658292, 0.4440999 ])]

In [19]:
# Calculate the performance of the portfolios
pf1 = portofolio_performance(returns, pf_weights[0])
pf2 = portofolio_performance(returns, pf_weights[1])
pf3 = portofolio_performance(returns, pf_weights[2])
pf4 = portofolio_performance(returns, pf_weights[3])
pf5 = portofolio_performance(returns, pf_weights[4])

In [20]:
# Print the portfolio performance
pf1

Ret       0.197989
Vol       0.198543
Sharpe    0.995697
AAPL      0.575851
IBM       0.236584
MSFT      0.187564
dtype: float64

In [21]:
# Create a portfolio data dictionary
data = {0:pf[:].values,
        1:pf1[:].values,
        2:pf2[:].values,
        3:pf3[:].values,
        4:pf4[:].values,
        5:pf5[:].values}

In [22]:
# Name the columns
columns = pf1.index

In [23]:
# Create a transposed dataframe
df = pd.DataFrame(data, columns).T

In [24]:
# Print the dataframe
df

,Ret,Vol,Sharpe,AAPL,IBM,MSFT
0,0.184298,0.190583,0.965448,0.500000,0.300000,0.200000
1,0.197989,0.198543,0.995697,0.575851,0.236584,0.187564
2,0.156000,0.178637,0.871601,0.325473,0.424744,0.249784
3,0.179091,0.187138,0.955396,0.456908,0.319041,0.224051
4,0.181704,0.186503,0.972660,0.267777,0.234356,0.497867
5,0.219051,0.202294,1.081352,0.489317,0.066583,0.444100


In [25]:
# Locate the maximum Sharpe ratio portfolio
max_sharpe_pf = df[df['Sharpe'] == df['Sharpe'].max()]

In [26]:
# Print the maximum Sharpe portfolio
max_sharpe_pf

,Ret,Vol,Sharpe,AAPL,IBM,MSFT
5,0.219051,0.202294,1.081352,0.489317,0.066583,0.4441


In [27]:
# Locate the maximum return portfolio
max_ret_pf = df[df['Ret'] == df['Ret'].max()]

In [28]:
# Print the maximum return portfolio
max_ret_pf

,Ret,Vol,Sharpe,AAPL,IBM,MSFT
5,0.219051,0.202294,1.081352,0.489317,0.066583,0.4441


In [29]:
# Locate the minimum volatility portfolio
min_vol_pf = df[df['Vol'] == df['Vol'].min()]

In [30]:
# Print the minimum volatiltiy portfolio
min_vol_pf

,Ret,Vol,Sharpe,AAPL,IBM,MSFT
2,0.156,0.178637,0.871601,0.325473,0.424744,0.249784


In [31]:
# Make a list of stocks
tickers = ['AAPL','IBM', 'MSFT', 'WMT']

In [32]:
# # Load stock prices
# df = web.DataReader(tickers,'yahoo','01/01/2010','10/18/2019')['Adj Close']

In [33]:
# # Save the data to a file
# df.to_csv('AAPL_MSFT_IBM_WMT.csv')

In [34]:
#  Load the data
df = pd.read_csv('AAPL_MSFT_IBM_WMT.csv', index_col='Date', parse_dates=True)

In [35]:
# Print full summary
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2466 entries, 2010-01-04 to 2019-10-18
Data columns (total 4 columns):
AAPL    2466 non-null float64
IBM     2466 non-null float64
MSFT    2466 non-null float64
WMT     2466 non-null float64
dtypes: float64(4)
memory usage: 96.3 KB


In [36]:
# Print the first few rows
df.head()

,AAPL,IBM,MSFT,WMT
Date,,,,
2010-01-04,26.601469,96.997330,24.360727,42.309753
2010-01-05,26.647457,95.825584,24.368599,41.888462
2010-01-06,26.223597,95.203102,24.219046,41.794838
2010-01-07,26.175119,94.873573,23.967175,41.818237
2010-01-08,26.349140,95.825584,24.132469,41.607594


In [37]:
# Print the last few rows
df.tail()

,AAPL,IBM,MSFT,WMT
Date,,,,
2019-10-14,235.163971,140.381943,139.076767,118.637856
2019-10-15,234.615631,141.330734,141.089920,118.996246
2019-10-16,233.668457,140.451126,139.933838,118.886742
2019-10-17,234.575745,132.692764,139.216278,119.304863
2019-10-18,235.702362,132.524750,136.944016,118.607994


In [38]:
# Calculate the daily returns
returns = df.pct_change().dropna()

In [39]:
# Print the first few rows
returns.head()

,AAPL,IBM,MSFT,WMT
Date,,,,
2010-01-05,0.001729,-0.012080,0.000323,-0.009957
2010-01-06,-0.015906,-0.006496,-0.006137,-0.002235
2010-01-07,-0.001849,-0.003461,-0.010400,0.000560
2010-01-08,0.006648,0.010035,0.006897,-0.005037
2010-01-11,-0.008821,-0.010470,-0.012720,0.016501


In [40]:
# Print the last few rows
returns.tail()

,AAPL,IBM,MSFT,WMT
Date,,,,
2019-10-14,-0.001439,-0.005043,-0.000931,-0.008899
2019-10-15,-0.002332,0.006759,0.014475,0.003021
2019-10-16,-0.004037,-0.006224,-0.008194,-0.000920
2019-10-17,0.003883,-0.055239,-0.005128,0.003517
2019-10-18,0.004803,-0.001266,-0.016322,-0.005841


In [41]:
# The number of periods
n_periods = 1

# Calculate the mean of n-period returns
mean = returns.mean() * n_periods

# Calculate the standard deviation of n-period returns
stdev = returns.std() * np.sqrt(n_periods)

In [42]:
# Print the the mean and standard deviation of returns
for ticker, ret, vol in zip(tickers, mean, stdev):
    p(f'{ticker} stock {n_periods}-day performance:')
    p(f'The mean daily return is {round(ret,6)}.')
    p(f'The standatd deviation of daily returns (volatitliy) is {round(vol,4)*100}%.\n')

AAPL stock 1-day performance:
The mean daily return is 0.001019.
The standatd deviation of daily returns (volatitliy) is 1.63%.

IBM stock 1-day performance:
The mean daily return is 0.000204.
The standatd deviation of daily returns (volatitliy) is 1.24%.

MSFT stock 1-day performance:
The mean daily return is 0.000804.
The standatd deviation of daily returns (volatitliy) is 1.44%.

WMT stock 1-day performance:
The mean daily return is 0.000478.
The standatd deviation of daily returns (volatitliy) is 1.09%.



In [43]:
# The number of periods
n_periods = 40

# Calculate the mean of n-period returns
mean = returns.mean() * n_periods

# Calculate the standard deviation of n-period returns
stdev = returns.std() * np.sqrt(n_periods)

In [44]:
# Print the the mean and standard deviation of returns
for ticker, ret, vol in zip(tickers, mean, stdev):
    p(f'{ticker} stock {n_periods}-day performance:')
    p(f'The mean daily return is {round(ret,6)}.')
    p(f'The standatd deviation of daily returns (volatitliy) is {round(vol,4)*100}%.\n')

AAPL stock 40-day performance:
The mean daily return is 0.040743.
The standatd deviation of daily returns (volatitliy) is 10.31%.

IBM stock 40-day performance:
The mean daily return is 0.008154.
The standatd deviation of daily returns (volatitliy) is 7.85%.

MSFT stock 40-day performance:
The mean daily return is 0.032179.
The standatd deviation of daily returns (volatitliy) is 9.120000000000001%.

WMT stock 40-day performance:
The mean daily return is 0.019122.
The standatd deviation of daily returns (volatitliy) is 6.92%.



In [45]:
# Print variance of returns
for ticker, var in zip(tickers, returns.var()):
    p(f'The variance of daily returns for {ticker} is {var}.')

The variance of daily returns for AAPL is 0.0002659402280278109.
The variance of daily returns for IBM is 0.00015391483338031566.
The variance of daily returns for MSFT is 0.00020778378812307306.
The variance of daily returns for WMT is 0.00011965542699554657.


In [46]:
# Print covariance of returns and itself
for ticker, cov in zip(tickers, np.cov(returns[tickers])):
    p(f'The covariance of daily returns for {ticker} is {np.cov(returns[ticker])}.')

The covariance of daily returns for AAPL is 0.0002659402280278106.
The covariance of daily returns for IBM is 0.0001539148333803158.
The covariance of daily returns for MSFT is 0.00020778378812307295.
The covariance of daily returns for WMT is 0.00011965542699554633.


In [47]:
# Calculate the variance-covariance matrix
cov_matrix = returns.cov()

# Print the variance-covariance matrix
cov_matrix

,AAPL,IBM,MSFT,WMT
AAPL,0.000266,0.000078,0.000108,0.000040
IBM,0.000078,0.000154,0.000088,0.000041
MSFT,0.000108,0.000088,0.000208,0.000049
WMT,0.000040,0.000041,0.000049,0.000120


In [48]:
# Calculate the correaltion matrix
corr_matrix = returns.corr()

# Print the correaltion matrix
corr_matrix

,AAPL,IBM,MSFT,WMT
AAPL,1.000000,0.385759,0.457803,0.225877
IBM,0.385759,1.000000,0.491558,0.299074
MSFT,0.457803,0.491558,1.000000,0.307862
WMT,0.225877,0.299074,0.307862,1.000000


In [49]:
# portfolio weights
weights = np.asarray([0.5, 0.3, 0.1, 0.1])

In [50]:
# Calculate portfolio performance
pf_perf = portofolio_performance(returns, weights)

In [51]:
# Print portfolio performance
pf_perf

Ret       0.176072
Vol       0.181343
Sharpe    0.969279
AAPL      0.500000
IBM       0.300000
MSFT      0.100000
WMT       0.100000
dtype: float64

We created a portfolio with manually selected asset weights. Next, we will create five new portfolios. Each new portfolio will be constructed using randomly selected asset weights. Then, we will compare the all of the portfolios.

In [52]:
# Set the random number generator
np.random.seed(54321)

# Determine the number of portfolios
n_portfolio = 8

# Create an empty list of portoflio weights
pf_weights = []

# Create portoflios with randomly weighted assets
for i in range(n_portfolio):
        
        # Randomly generate a weight for each asset
        weights = np.random.random(len(tickers))
        
        # Normalize the weights to set the sum to 1
        weights /= sum(weights)
        
        # Attach each weights vecotr to the list of portfolio weights
        pf_weights.append(weights)

In [53]:
# Print list of weights vectors
pf_weights

[array([0.33065754, 0.22626498, 0.28719197, 0.15588551]),
 array([0.29840386, 0.22720921, 0.04706245, 0.42732448]),
 array([0.43680535, 0.04514181, 0.48100594, 0.0370469 ]),
 array([0.23852248, 0.29376561, 0.18852578, 0.27918614]),
 array([0.30591352, 0.34134841, 0.04753332, 0.30520474]),
 array([0.43034204, 0.03870507, 0.10889037, 0.42206251]),
 array([0.38211494, 0.06221279, 0.40034242, 0.15532986]),
 array([0.31476347, 0.04552468, 0.38411885, 0.255593  ])]

In [54]:
# Calcualte portfolio performance of randomly weighted assets
pf1 = portofolio_performance(returns,pf_weights[0])
pf2 = portofolio_performance(returns,pf_weights[1])
pf3 = portofolio_performance(returns,pf_weights[2])
pf4 = portofolio_performance(returns,pf_weights[3])
pf5 = portofolio_performance(returns,pf_weights[4])
pf6 = portofolio_performance(returns,pf_weights[5])
pf7 = portofolio_performance(returns,pf_weights[6])
pf8 = portofolio_performance(returns,pf_weights[7])

In [55]:
# Create a data dictionary
data = {0:pf_perf[:].values,
        1:pf1[:].values,
        2:pf2[:].values,
        3:pf3[:].values,
        4:pf4[:].values,
        5:pf5[:].values,
        6:pf6[:].values,
        7:pf7[:].values,
        8:pf8[:].values}

In [56]:
# Name the columns
columns = pf1.index

In [57]:
# Create a transposed dataframe
df = pd.DataFrame(data, columns).T

In [58]:
# Print the performance of the portfolios
df

,Ret,Vol,Sharpe,AAPL,IBM,MSFT,WMT
0,0.176072,0.181343,0.969279,0.500000,0.300000,0.100000,0.100000
1,0.173498,0.167930,1.031371,0.330658,0.226265,0.287192,0.155886
2,0.149286,0.150206,0.991876,0.298404,0.227209,0.047062,0.427324
3,0.216415,0.197368,1.094987,0.436805,0.045142,0.481006,0.037047
4,0.148168,0.152768,0.967925,0.238522,0.293766,0.188526,0.279186
5,0.142461,0.153997,0.923141,0.305914,0.341348,0.047533,0.305205
6,0.185369,0.164293,1.126460,0.430342,0.038705,0.108890,0.422063
7,0.201151,0.179765,1.117299,0.382115,0.062213,0.400342,0.155330
8,0.191795,0.169073,1.132615,0.314763,0.045525,0.384119,0.255593


In [59]:
# Calculate the maximum Sharpe Ratio portfolio
max_sharpe_pf = df[df['Sharpe'] == df['Sharpe'].max()]

# Print the maximum Sharpe ratio
max_sharpe_pf

,Ret,Vol,Sharpe,AAPL,IBM,MSFT,WMT
8,0.191795,0.169073,1.132615,0.314763,0.045525,0.384119,0.255593


In [60]:
# Calculate the maximum returns portfolio
max_ret_pf = df[df['Ret'] == df['Ret'].max()]

# Print the maximum returns portfolio
max_ret_pf

,Ret,Vol,Sharpe,AAPL,IBM,MSFT,WMT
3,0.216415,0.197368,1.094987,0.436805,0.045142,0.481006,0.037047


In [61]:
# Calculate the minimum volatility portfolio
min_vol_pf = df[df['Vol'] == df['Vol'].min()]

# Print the minimum volatility portfolio
min_vol_pf

,Ret,Vol,Sharpe,AAPL,IBM,MSFT,WMT
2,0.149286,0.150206,0.991876,0.298404,0.227209,0.047062,0.427324


We have created and investigated many portfolios. Each portfolio is made of n-assets stocks, each of different weights. One portfolio was created by manually selecting the weight of each asset. A few portfolios were created by randomly selecting the weights of each asset. Of the portfolios, we found the highest Sharpe Ratio, highest returns, and lowest volatility portfolios. We will create many more portfolios of randomly generated asset weights, and find the best portfolios.